In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
np.set_printoptions(precision=3, suppress=True)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Dataset 

source: https://www.kaggle.com/datasets/wkirgsn/electric-motor-temperature


In [ ]:
train_df = pd.read_csv("measures_v2.csv")



In [ ]:
train_df.isna().sum()

There are no unknown values

In [ ]:
train_dataset = train_df.sample(frac=0.9, random_state=0)
test_dataset = train_df.drop(train_dataset.index)

Train/test split 90/10

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

pm-Permanent magnet temperature will be the target variable


In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('pm')
test_labels = test_features.pop('pm')

### Normalization Layer


In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
normalizer.adapt(np.array(train_features))

In [ ]:
print(normalizer.mean.numpy().transpose())

In [ ]:
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('One example:', first.transpose())
  print('Normalized:', normalizer(first).numpy().transpose())

In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])
linear_model.summary()

In [ ]:
linear_model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:

history = linear_model.fit(
    train_features,
    train_labels,
    epochs=100,
    verbose=1,
    validation_split = 0.1)

In [41]:
linear_model.predict(train_features[:10])

1/1 [==============================] - 0s 80ms/step


array([[ 42.176],
       [ 75.865],
       [ 56.804],
       [104.685],
       [ 85.869],
       [ 86.762],
       [ 74.645],
       [ 69.256],
       [ 69.006],
       [ 51.506]], dtype=float32)

In [40]:
evaluation=linear_model.evaluate( test_features, test_labels, verbose=1)

4159/4159 [==============================] - 13s 3ms/step - loss: 5.2874
